# Doublet-Source Panel Method

In [1]:
include("../src/PanelMethods.jl")
include("../src/FoilParametrization.jl")

using .PanelMethods: DoubletPanel2D, Uniform2D, grid_data, aero_coefficients
using .FoilParametrization: read_foil, cosine_foil, kulfan_CST
using BenchmarkTools
using Plots
plotly()

┌ Info: Precompiling PlotlyBase [a03496cd-edff-5a9b-9e67-9cda94a718b5]
└ @ Base loading.jl:1278


Plots.PlotlyBackend()

In [5]:
alpha_u = [0.1, 0.3, 0.2, 0.15, 0.2]
alpha_l = [-0.1, -0.1, -0.1, -0.001, -0.02]
alphas = [alpha_u alpha_l]
dzs = (1e-4, 1e-4)
cst_foil = kulfan_CST(alphas, dzs, 0.2, 80);
plot(cst_foil[:,1], cst_foil[:,2], aspect_ratio=:equal)

<!DOCTYPE html>
 
 
 Plots.jl

### Setup

In [6]:
panels = reverse([ DoubletPanel2D((xs, ys), (xe, ye)) for (xs, ys, xe, ye) ∈ (collect ∘ eachrow)([cst_foil[2:end,:] cst_foil[1:end-1,:]]) ], dims = 1)
uniform = Uniform2D(5.0, 5.0);

In [7]:
@time (cl, kcl, cps) = aero_coefficients(panels, uniform)
println("Lift Coefficient: $cl, $kcl")

  0.010070 seconds (28.36 k allocations: 3.089 MiB)
Lift Coefficient: 0.8077255950117708, 0.80282314355495
